In [1]:
import os
import torch 
import numpy
import pandas as pd 
import cudf
import cupy
import nvtabular as nvt

### Create random input data similar to pre-processed Yoochoose dataset structure 

In [2]:
NUM_ROWS = 10000
session_length = 20
batch_size = 100
inputs = {
    'session_id': numpy.random.randint(70000, 80000, NUM_ROWS),
    'day': numpy.random.randint(1, 10, NUM_ROWS),
    'item_id': numpy.random.randint(1, 51996, NUM_ROWS),
    'category': numpy.random.randint(0, 332, NUM_ROWS),
    'timestamp/age_days': numpy.random.uniform(0, 1, NUM_ROWS),
    'timestamp/weekday/sin' : numpy.random.uniform(0, 1, NUM_ROWS),
    'purchase': numpy.random.randint(0, 2, NUM_ROWS)
    }
random_data = cudf.DataFrame(inputs)
random_data

,session_id,day,item_id,category,timestamp/age_days,timestamp/weekday/sin,purchase
0,76747,6,22388,72,0.288054,0.533836,1
1,71167,9,47833,221,0.550444,0.116814,1
2,73963,4,22239,159,0.490460,0.485789,0
3,79271,5,3361,155,0.206031,0.062344,0
4,72333,9,29085,312,0.572049,0.032932,0
...,...,...,...,...,...,...,...
9995,72993,6,42526,110,0.471787,0.002256,1
9996,73604,3,19149,326,0.987786,0.858587,0
9997,75048,2,26950,122,0.136803,0.244733,0
9998,76540,5,23454,82,0.961493,0.404627,1


### NVTabular workflow

- #TODO : Change the workflow using tagging API once it is finalized  

In [3]:
# Define Groupby Workflow
groupby_features = list(inputs.keys()) >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list"],
        "category": ["list"],     
        "day": ["first"],
        "purchase": ["first"],
        "timestamp/age_days": ["list"],
        'timestamp/weekday/sin': ["list"],
        },
    name_sep="-")
# Trim sessions to first 20 items 
groupby_features_nonlist = [x for x in groupby_features.selector if '-list' not in x]
groupby_features_nonlist
groupby_features_trim = ((groupby_features - groupby_features_nonlist)) >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')

workflow = nvt.Workflow(groupby_features + groupby_features_trim )
dataset = nvt.Dataset(random_data, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

/nvtabular/nvtabular/workflow/node.py:43: FutureWarning: The `["a", "b", "c"] >> ops.Operator` syntax for creating a `ColumnGroup` has been deprecated in NVTabular 21.09 and will be removed in a future version.
  warnings.warn(


In [4]:
sessions_gdf.head(3)

,purchase-first,timestamp/age_days-list,session_id,timestamp/weekday/sin-list,item_id-list,category-list,day-first,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim,item_id-list_trim,category-list_trim
0,0,"[0.9162040642342256, 0.12168169060280332]",70000,"[0.1445256999821507, 0.34874245458422604]","[17689, 5731]","[272, 180]",3,"[0.9162040642342256, 0.12168169060280332]","[0.1445256999821507, 0.34874245458422604]","[17689, 5731]","[272, 180]"
1,0,[0.3648727333239704],70001,[0.2666721097634238],[31982],[162],4,[0.3648727333239704],[0.2666721097634238],[31982],[162]
2,1,"[0.7997741673137783, 0.7381101537531506]",70002,"[0.32939338781118777, 0.04611238136930851]","[36459, 27061]","[134, 276]",1,"[0.7997741673137783, 0.7381101537531506]","[0.32939338781118777, 0.04611238136930851]","[36459, 27061]","[134, 276]"


- We can save the workflow.

In [5]:
workflow.save('workflow_inference_test')

### Export pre-processed data by day 

In [6]:
# Convert to a Dataset and write out hive-partitioned data to disk
nvt_output_path_tmp ='./output_nvt_tmp/'
PARTITION_COL = 'day-first'
nvt.Dataset(sessions_gdf).to_parquet(nvt_output_path_tmp, partition_on=[PARTITION_COL])

In [7]:
OUTPUT_FOLDER = "./preproc_sessions_by_day_ts/"
!mkdir -p $OUTPUT_FOLDER
days_folders = [f for f in sorted(os.listdir(nvt_output_path_tmp)) if f.startswith(PARTITION_COL)]
for day_folder in days_folders:
    df = cudf.read_parquet(os.path.join(nvt_output_path_tmp, day_folder))
    out_folder = os.path.join(OUTPUT_FOLDER, day_folder.replace('day-first=', ''))
    os.makedirs(out_folder, exist_ok=True)
    df.to_parquet(os.path.join(out_folder, 'train.parquet'))
    
    random_values = cupy.random.rand(len(df))
    
    #Extracts 10% for valid and test set. Those sessions are also in the train set, but as evaluation
    #happens only for the subsequent day of training, that is not an issue, and we can keep the train set larger.
    valid_set = df[random_values <= 0.10]
    valid_set.to_parquet(os.path.join(out_folder, 'valid.parquet'))
    
    test_set = df[random_values >= 0.90]
    test_set.to_parquet(os.path.join(out_folder, 'test.parquet'))

# Transformers4rec model  

In [8]:
import torch 
import transformers4rec.torch as torch4rec
from transformers4rec.torch import TabularSequenceFeatures, MLPBlock, SequentialBlock, Head, TransformerBlock

from transformers4rec.utils.schema import DatasetSchema

from transformers4rec.torch.head import NextItemPredictionTask

from transformers4rec.config import transformer
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt

- Manually set the schema 

In [9]:
# Define schema object to pass it to the SequentialTabularFeatures
schema = DatasetSchema.from_schema("schema.pb")

### Define the sequential input module

Below we define our `input` bloc using [`SequentialTabularFeatures` class](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/features/sequential.py). The `from_schema` module directly parse schema and accepts categorical and continuous sequential inputs and supports data augmentation, data aggregation, `sequential-concat` and `elementwise-sum` aggregations, the projection of the interaction embeddings and the masking tasks.

`max_sequence_length` defines the maximum sequence length of our sequential input, and if `continuous_projection` argument is set,  all numerical features are concatenated and projected by a number of MLP layers.

In [10]:
inputs = TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=20,
        continuous_projection=64,
        d_output=100,
        masking="causal",
    )

inputs.masking.device = 'cuda'

### End-to-end session-based Transformer-based model for item prediction:

- LM task + HF Transformer architecture + Next item-prediction task. 
- We build a [T4RecConfig](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/config/transformer.py#L8) class to update the config class of the transformer architecture with the specified arguments, then load the related model. Here we use it to instantiate an XLNET model according to the  arguments (d_model, n_head, etc.), defining the model architecture.
- [TransformerBlock](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/block/transformer.py#L37) class is created to support HF Transformers for session-based and sequential-based recommendation models.
- [NextItemPredictionTask](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/head.py#L212) is the class to support next item prediction task.

In [11]:
# case-1: Define XLNetConfig class and set default parameters 

transformer_config = transformer.XLNetConfig.build(
    d_model=64, n_head=4, n_layer=2, total_seq_length=20
)

body = torch4rec.SequentialBlock(
    inputs, torch4rec.MLPBlock([64]), torch4rec.TransformerBlock(transformer=transformer_config, masking=inputs.masking)
)

head = torch4rec.Head(
    body,
    torch4rec.NextItemPredictionTask(weight_tying=True, hf_format=True, metrics=[NDCGAt(top_ks=[5,10, 1000], labels_onehot=True),
                                                                                 AvgPrecisionAt(top_ks=[5,10, 1000],labels_onehot=True)]),
    inputs=inputs,
)
model = torch4rec.Model(head)

# Train the model 

## Non-incremental training and evaluation

- Set Training arguments

In [12]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
#Set argumentd for training 
train_args = T4RecTrainingArguments(local_rank = -1, dataloader_drop_last = True,
                  report_to = [], debug = ["r"], gradient_accumulation_steps = 32,
                  per_device_train_batch_size = 512, per_device_eval_batch_size = 512,
                  output_dir = ".", use_legacy_prediction_loop = False)

- Build dataloader 

In [13]:
import glob
from transformers4rec.torch.utils.data_utils import PyarrowDataLoaderBuilder, NVTDataLoaderBuilder
train_data_paths = glob.glob("./preproc_sessions_by_day_ts/*/train.parquet")
eval_data_paths = glob.glob("./preproc_sessions_by_day_ts/*/valid.parquet")
# Specify sparse features
SESSION_LENGTH_MAX = 20
x_cat_names, x_cont_names = ['item_id-list_trim', 'category-list_trim'], ['timestamp/weekday/sin-list_trim','timestamp/age_days-list_trim']
sparse_features_max = {
    fname: SESSION_LENGTH_MAX
    for fname in x_cat_names + x_cont_names
}

In [14]:
train_nvt_loader = NVTDataLoaderBuilder.from_schema(schema,
                                paths_or_dataset=train_data_paths, 
                                batch_size=train_args.per_device_train_batch_size,
                                max_sequence_length=20,
                                drop_last=True,
                                shuffle=True,
                                sparse_as_dense=True,
                                sparse_names=x_cat_names+x_cont_names,
                                sparse_max=sparse_features_max
                                )


eval_nvt_loader = NVTDataLoaderBuilder.from_schema(schema,
                                paths_or_dataset=eval_data_paths, 
                                batch_size=train_args.per_device_train_batch_size,
                                max_sequence_length=20,
                                drop_last=True,
                                shuffle=True,
                                sparse_as_dense=True,
                                sparse_names=x_cat_names+x_cont_names,
                                sparse_max=sparse_features_max
                                )

- Define Trainer

In [15]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=train_args,
    compute_metrics=True,
    train_dataloader=train_nvt_loader,
    eval_dataloader=eval_nvt_loader,
)

- Launch one full train and evaluation : 

In [16]:
trainer.reset_lr_scheduler()
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=15.015725453694662, metrics={'train_runtime': 2.159, 'train_samples_per_second': 1.39, 'total_flos': 0.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2700804096, 'init_mem_gpu_alloc_delta': 14147584, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 23719936, 'train_mem_gpu_alloc_delta': 42733056, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 441208832})

In [17]:
train_metrics = trainer.evaluate(metric_key_prefix='eval')
for key in sorted(train_metrics.keys()):
    print("  %s = %s" % (key, str(train_metrics[key])))

/opt/conda/lib/python3.8/site-packages/transformers/trainer_pt_utils.py:366: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Not all data has been set. Are you sure you passed all values?
Not all data has been set. Are you sure you passed all values?


  epoch = 3.0
  eval_avgprecisionat_@10 = 0.0
  eval_avgprecisionat_@1000 = 7.355372e-05
  eval_avgprecisionat_@5 = 0.0
  eval_loss = 33.65076446533203
  eval_mem_cpu_alloc_delta = 1916928
  eval_mem_cpu_peaked_delta = 0
  eval_mem_gpu_alloc_delta = 92160
  eval_mem_gpu_peaked_delta = 541213184
  eval_ndcgat_@10 = 0.0
  eval_ndcgat_@1000 = 0.0033141105
  eval_ndcgat_@5 = 0.0
  eval_runtime = 1.2938
  eval_samples_per_second = 395.733


## Incremental training and evaluation : 

- Set Trainer 

In [18]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
#Set argumentd for training 
train_args = T4RecTrainingArguments(local_rank = -1, dataloader_drop_last = True,
                  report_to = [], debug = ["r"], gradient_accumulation_steps = 32,
                  per_device_train_batch_size = 512, per_device_eval_batch_size = 32,
                  output_dir = ".", use_legacy_prediction_loop = False)

# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=train_args,
    compute_metrics=True,
)

- Launch incremental training and evaluation : 

In [19]:
start_time_window_index = 1
final_time_window_index = 3

for time_index in range(start_time_window_index, final_time_window_index):
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = glob.glob(f"./preproc_sessions_by_day_ts/{time_index_train}/train.parquet")
    eval_paths = glob.glob(f"./preproc_sessions_by_day_ts/{time_index_eval}/valid.parquet")
    # Build data-loader
    train_nvt_loader = NVTDataLoaderBuilder.from_schema(schema,
                                paths_or_dataset=train_paths, 
                                batch_size=train_args.per_device_train_batch_size,
                                max_sequence_length=20,
                                drop_last=True,
                                shuffle=True,
                                sparse_as_dense=True,
                                sparse_names=x_cat_names+x_cont_names,
                                sparse_max=sparse_features_max
                        )
    
    # Set dataloaders
    trainer.set_train_dataloader(train_nvt_loader)    
    trainer.reset_lr_scheduler()
    trainer.train()
    
    eval_nvt_loader = NVTDataLoaderBuilder.from_schema(schema,
                            paths_or_dataset= eval_paths,
                            batch_size=train_args.per_device_eval_batch_size,
                            max_sequence_length=20,
                            drop_last=True,
                            shuffle=True,
                            sparse_as_dense=True,
                            sparse_names=x_cat_names+x_cont_names,
                            sparse_max=sparse_features_max
                            )
    trainer.set_eval_dataloader(eval_nvt_loader)
    # Evaluate on the following day
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print("Eval results for day %s are:\n\t" %time_index_eval)
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key])))   
    trainer.wipe_memory()

Step,Training Loss


Not all data has been set. Are you sure you passed all values?
Not all data has been set. Are you sure you passed all values?


Eval results for day 2 are:
	
 epoch = 3.0
 eval_avgprecisionat_@10 = 0.0
 eval_avgprecisionat_@1000 = 3.850004e-05
 eval_avgprecisionat_@5 = 0.0
 eval_loss = 32.82271766662598
 eval_mem_cpu_alloc_delta = -102400
 eval_mem_cpu_peaked_delta = 135168
 eval_mem_gpu_alloc_delta = -87040
 eval_mem_gpu_peaked_delta = 37295104
 eval_ndcgat_@10 = 0.0
 eval_ndcgat_@1000 = 0.0027331354
 eval_ndcgat_@5 = 0.0
 eval_runtime = 2.0322
 eval_samples_per_second = 31.493


Step,Training Loss


Not all data has been set. Are you sure you passed all values?
Not all data has been set. Are you sure you passed all values?


Eval results for day 3 are:
	
 epoch = 3.0
 eval_avgprecisionat_@10 = 0.0
 eval_avgprecisionat_@1000 = 7.1705144e-05
 eval_avgprecisionat_@5 = 0.0
 eval_loss = 34.384437561035156
 eval_mem_cpu_alloc_delta = 12288
 eval_mem_cpu_peaked_delta = 16384
 eval_mem_gpu_alloc_delta = -80896
 eval_mem_gpu_peaked_delta = 34725888
 eval_ndcgat_@10 = 0.0
 eval_ndcgat_@1000 = 0.003087417
 eval_ndcgat_@5 = 0.0
 eval_runtime = 2.0897
 eval_samples_per_second = 30.626
